### Word2Vec
- 성공적으로 단어를 벡터로 표현하는 방법에는 크게 '통계 기반 기법'과 '추론 기반 기법' 두 가지가 존재하며, 모두 분포 가설을 배경으로 함

#### 통계 기반 기법의 문제점
- 단어의 동시발생 행렬을 만들고 그 행렬에 SVD(특이값 분해)를 적용하여 밀집벡터(단어의 분산 표현)을 얻는 방법으로 단어를 표현하였지만 대규모 말뭉치를 다룰 때 문제가 발생함
 * 현업에서 다루는 말뭉치의 어휘 수는 상당함
  > - 어휘의 수가 100만개라고 가정할 때, 통계 기반 기법에서는 '100만 x 100만'이라는 행렬을 만듦
  > - SVD를 nxn행렬에 적용하는 비용은 O(n**3)으로, 이는 슈퍼컴퓨터를 동원하여도 처리할 수 없는 수준
- 신경망의 경우 미니배치로 학습하는 것이 일반적이지만 통계 기반 기법은 학습 데이터를 한꺼번에 처리하는 '배치 학습'을 이용
 * 반면에 추론 기반 기법은 학습 데이터의 일부를 사용하여 순차적으로 학습하는 '미니배치 학습'을 이용

#### 추론 기반 기법 개요
- 추론 기반 기법은 '추론'이 주된 작업으로 주변 단어(맥락)이 주어졌을 때, '?(알고자하는 단어)'에 무슨 단어가 들어가는지를 추측하는 작업임
- 추론 기반 기법을 이용하는 모델은 맥락 정보를 입력받아 (출현할 수 있는) 각 단어의 출현 확률을 출력함
 * 말뭉치를 사용해 모델이 올바른 추측을 내놓도록 학습시키며, 분포 가설에 근거하는 '단어의 동시발생 가능성'을 얼마나 잘 모델링하는가가 중요한 연구 주제임

#### 신경망에서의 단어 처리
- 신경망은 단어를 있는 그대로 처리할 수 없어 단어를 '고정 길이의 벡터'로 변환해야 함
- 대표적인 방법이 단어를 원핫(one-hot) 표현(or 벡터)으로 변환하는 것임
 * 원핫 표현이란 벡터의 원소 중 하나만 1이고 나머지는 모두 0인 벡터를 뜻함

In [1]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])   ### 입력
W = np.random.randn(7, 3)   ### 가중치
h = np.matmul(c, W)   ### 중간 노드
print(h)

[[ 0.67031746  1.10060933 -0.516402  ]]


#### 단순한 Word2Vec
- Word2Vec에서 사용되는 신경망은 크게 CBoW와 Skip-gram이 있음

#### CBoW(Continuous Bag-of-Words)
- CBoW 모델은 맥락으로부터 타깃을 추측하는 용도의 신경망임
 * 여기에서 '타깃'은 중앙 단어이고, 그 주변 단어들이 '맥락'임
- CBoW 모델의 입력층의 갯수는 맥락에 포함시킬 단어의 갯수와 같음
 * 입력층의 갯수가 N개라면, 은닉층에서는 입력층으로부터 변환된 값들의 평균을 취해주면 됨
- 은닉층의 뉴런 수를 입력층의 뉴런 수보다 적게해주는 것이 중요함
 * 단어 예측에 필효한 정보를 간결하게 담기 위해서는 은닉층의 뉴런 수가 더 적은 것이 좋음
- CBoW 모델의 추론 처리 구현

In [2]:
from common.layers import MatMul

## 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

### 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

### 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

### 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)

[[-0.34046521 -1.67298867  0.74867183 -0.76513088  2.27448166  1.40992561
  -0.53276856]]


- CBoW 모델의 학습
 * 모델 학습에서는 올바른 예측을 할 수 있도록 가중치를 조정을 해야 함
 * CBoW 모델은 단어 출현 패턴을 학습 시 사용한 말뭉치로부터 학습이 되고, 말뭉치가 다르면 학습 후 얻게 되는 단어의 분산 표현도 달라짐
 * 따라서, 말뭉치의 종류에 따라 얻게되는 단어의 분산 표현이 크게 다를 수 있음
  > '스포츠'가사만을 사용하는 경우 vs '음악'기사만을 사용하는 경우
 * CBoW 모델은 다중 클래스 분류를 수행하는 신경망으로, 신경망을 학습하기 위해서 소프트맥스와 교차 엔트로피 오차를 이용하면 됨
  > 소프트 함수를 이용해 점수를 확률로 변환하고, 그 확률과 정답 레이블로부터 교차 엔트로피 오차를 구한 수 그 값을 손실로 사용해 학습을 진행함

- Word2Vec의 가중치와 분산 표현
 * Word2Vec에서 사용되는 신경망에는 입력 측 완전연결계층의 가중치(W_in)과 출력 측 완전연결계층의 가중치(W_out) 두 가지가 있음
  > - 입력 측 가중치 W_in의 각 행이 각 단어의 분산 표현에 해당됨
  > - 출력 측 가중치 W_out은 단어의 의미가 인코딩된 벡터가 저장되어 있다고 할 수 있음
 * 최종적으로 이용하는 단어의 분산 표현의 선택지는 세가지가 존재함
  > A. 입력 측의 가중치만 이용한다.
  > B. 출력 측의 가중치만 이용한다.
  > C. 양쪽 가중치를 모두 이용한다.
 * Skip-gram 모델에서는 입력 측의 가중치만 이용하는 A안을 보편적으로 사용함
 * 많은 연구에서도 출력 측 가중치는 버리고 입력 측 가중치 W_in만을 최종 단어의 분산 표현으로 이용함

- 맥락과 타깃
 * 신경망의 입력 값은 '맥락', 정답 레이블은 '타깃(맥락에 둘러싸인 중앙의 단어)'이 이용됨
 * 즉, 신경망에 '맥락'을 입력했을 때 '타깃'이 출현할 확률을 높이는 것이 학습 목표라고 할 수 있음

In [3]:
# from common.util import preprocess

### 말뭉치 텍스트를 단어 ID로 변환하는 함수(전처리)
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    corpus = np.array([word_to_id[w] for w in words])
    return corpus, word_to_id, id_to_word


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [4]:
# from common.util import create_contexts_target

### 말뭉치로부터 맥락과 타깃을 만드는 함수
def create_contexts_target(corpus, window_size=1):
    target = corpus[window_size:-window_size]
    context = []
    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size+1):
            if t == 0:
                continue
            cs.append(corpus[idx+t])
        context.append(cs)
    return np.array(context), np.array(target)

contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
[1 2 3 4 1 5]


In [5]:
# from common.util import convert_one_hot

### 원핫 표현으로 변환
def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0]
    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1
    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1
    return one_hot


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)
print(target)
print(contexts)

[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
[[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]


- CBoW 모델 구현

In [6]:
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
    ### 초기화 매서드 설정
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size
        
        ### 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')
        
        ### 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()
        
        ### 모든 가중치와 기울기를 리스트에 저장
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        ### 인스턴스 변수에 단어의 분산 표현을 저장
        self.word_vecs = W_in
    
    ### 순전파 매서드 설정
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = 0.5 * (h0 + h1)
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss
    
    ### 역전파 매서드 설정
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer0.backward(da)
        self.in_layer1.backward(da)
        return None
    
    
from common.trainer import Trainer
from common.optimizer import Adam
# from simple_cbow import SimpleCBOW
# from common.util import preprocess, create_contexts_target, convert_one_hot

window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

| 에폭 1 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 2 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 3 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 4 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 5 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 6 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 7 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 8 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 9 |  반복 1 / 2 | 시간 0[s] | 손실 1.95
| 에폭 10 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 11 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 12 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 13 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 14 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 15 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 16 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 17 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 18 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 19 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 20 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 21 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 22 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 23 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 24 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 25 |  반복 1 / 2 | 시간 0[s] | 손실 1.94
| 에폭 26 |

<Figure size 640x480 with 1 Axes>

In [7]:
word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])

you [-0.9346086   1.7752006  -0.89905334  0.8294531  -1.0793817 ]
say [ 1.3264676  -0.4541395   0.31451392 -1.2769911   0.29507965]
goodbye [-0.90505886 -0.4009965  -1.1304864   1.0613316  -0.98904806]
and [ 1.0822923 -1.2898159  1.3267044 -1.0638983 -1.7040696]
i [-0.90687734 -0.4023581  -1.1255732   1.070751   -0.9922357 ]
hello [-0.9249057   1.7806956  -0.90953267  0.82565904 -1.0803144 ]
. [ 1.1757185  1.3413272 -1.4591327 -1.1341753  1.2334932]


#### Word2Vec 보충
- CBoW 모델 : 맥락이 여러 개 있고, 그 여러 맥락으로부터 중앙의 단어(타깃)을 추측함
- Skip-gram 모델 : 중앙의 단어(타깃)로부터 주변의 여러 단어(맥락)를 추측함
- CBoW 모델 vs Skip-gram 모델
 * 단어 분산 표현의 정밀도 면에서 skip-gram 모델의 경우가 더 많음
 * 말뭉치가 커질수록 저빈도 단어나 유추 문제의 성능 면에서 skip-gram 모델이 더 뛰어난 경향이 있음
 * 학습 속도 면에서는 CBoW가 더 빠름
  > 이는 skip-gram 모델은 손실을 맥락의 수만큼 구해야 해서 계산 비용이 그만큼 커짐

#### Conclusion
- 추론 기반 기법은 추측하는 것이 목적이며, 그 부산물로 단어의 분산 표현을 얻을 수 있음
- Word2Vec은 추론 기반 기법이며, Skip-gram 모델과 CBoW 모델을 제공함
- CBoW 모델은 여러 단어(맥락)로부터 하나의 단어(타깃)를 추측하고, Skip-gram 모델은 하나의 단어(타깃)로부터 다수의 단어(맥락)를 추측함
- Word2Vec은 가중치를 다시 학습할 수 있으므로 단어의 분산 표현 갱신이나 새로운 단어 추가를 효율적으로 수행할 수 있음